In [27]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
from tqdm.notebook import tqdm
import pandas as pd
import re
import sqlite3

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [3]:
ADAPTER = "models/gemma3_1b_text2sql_lora"

In [4]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('CUDA is available. Using GPU.')
else:
    device = torch.device('cpu')
    print('CUDA is not available. Using CPU.')

CUDA is available. Using GPU.


In [7]:
tokenizer = AutoTokenizer.from_pretrained(ADAPTER, device_map=device)

In [8]:
base_model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it").to(device)

In [11]:
model = PeftModel.from_pretrained(base_model, ADAPTER)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForCausalLM(
      (model): Gemma3TextModel(
        (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 1152, padding_idx=0)
        (layers): ModuleList(
          (0-25): 26 x Gemma3DecoderLayer(
            (self_attn): Gemma3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1152, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1152, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
        

In [13]:
model.device

device(type='cuda', index=0)

In [14]:
def format_test_example(row) -> str:
    return f"""<|system|>
Ты — интеллектуальный текстовый помощник, преобразующий команды пользователя в корректные SQL-запросы на языке SQLite.

Ты всегда работаешь с таблицей `transactions`, имеющей следующую структуру:

- id (INTEGER, PRIMARY KEY)
- user_id (TEXT)
- type (TEXT: 'income' или 'expense')
- category (TEXT)
- amount (REAL)
- date (TIMESTAMP, по умолчанию date('now', 'localtime'))

Описание:
- `income` — это доход
- `expense` — это трата

Твоя задача — по команде пользователя с его ID сгенерировать корректный SQL-запрос к этой таблице.

Важно:
- Используй только SQLite-синтаксис, без пояснений.
- Не пиши комментарии.
- Не используй несуществующие поля (например, `note`).
- Поле `date` можно указывать через date('now', '-N day') или опустить (будет по умолчанию).

Пример:
---
Пользователь с ID user_1 дал команду:
"Добавь трату 500 рублей на еду вчера"

Ответ:
INSERT INTO transactions (user_id, type, category, amount, date)
VALUES ('user_1', 'expense', 'еда', 500, date('now', '-1 day'));
---
</s>
<|user|>
Пользователь с ID {row["user_id"]} дал команду:
"{row["user_command"]}"
</s>
<|assistant|>
"""

In [15]:
def clean_sql(text):
    """
    Удаляет обёртки ```sql ... ``` и специальные токены вроде </s>, <|...|>.
    """
    if not isinstance(text, str):
        return ""

    # Убираем блоки ```sql и ```
    text = re.sub(r"```sql\s*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"```", "", text)

    # Убираем спецтокены LLM
    text = re.sub(r"</s>", "", text)
    text = re.sub(r"<\|.*?\|>", "", text)

    return text.strip()

In [16]:
def generate_sql_from_df(test_df, model, tokenizer, max_new_tokens=128):
    predicted_sqls = []

    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        prompt = format_test_example(row)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=0.1,
                pad_token_id=tokenizer.eos_token_id
            )

        decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
        sql = decoded.split("<|assistant|>")[-1].strip()
        predicted_sqls.append(clean_sql(sql))

    test_df["predicted_sql"] = predicted_sqls
    return test_df

In [18]:
test_df = pd.read_csv("data/fin_ass_test.csv")

In [25]:
test_df.shape

(1000, 3)

In [37]:
temp = test_df.sample(100).copy()

In [38]:
temp = generate_sql_from_df(temp, model, tokenizer)

  0%|          | 0/100 [00:00<?, ?it/s]

In [39]:
temp

,user_id,user_command,gold_sql,predicted_sql
640,user_1,Удалите мои последние траты на одежда,DELETE FROM transactions\nWHERE user_id = 'use...,"INSERT INTO transactions (user_id, type, categ..."
273,user_1,Удалите мои последние траты на фриланс,DELETE FROM transactions\nWHERE user_id = 'use...,"Change adjustment = 'expense',\nTIMESTAMP = '2..."
630,user_1,Удалите мои последние траты на транспорт,DELETE FROM transactions\nWHERE user_id = 'use...,"INSERT INTO transactions (user_id, type, categ..."
62,user_2,Удалите мои последние траты на еда,DELETE FROM transactions\nWHERE user_id = 'use...,"INSERT INTO transactions (user_id, type, categ..."
563,user_3,Сколько я потратил на развлечения за последние...,SELECT SUM(amount) FROM transactions\nWHERE us...,SELECT SUM(amount
...,...,...,...,...
163,user_5,Какова средняя сумма моих трат на зарплата за ...,SELECT AVG(amount) FROM transactions\nWHERE us...,SELECT AVG(amount) FROM transactions\nWHERE us...
634,user_1,Покажи мои траты на медицина между '2024-01-01...,SELECT * FROM transactions\nWHERE user_id = 'u...,"INSERT INTO transactions (user_id, type, categ..."
209,user_3,Покажи мои расходы по категориям за последние ...,"SELECT category, SUM(amount) FROM transactions...","SELECT category, SUM(amount) FROM transactions..."
489,user_5,Покажи мои расходы по категориям за последние ...,"SELECT category, SUM(amount) FROM transactions...","SELECT category, SUM(amount) FROM transactions..."


In [40]:
def normalize_sql(sql: str) -> str:
    return sql.strip().lower().rstrip(";")

def exact_match(pred: str, gold: str) -> bool:
    return normalize_sql(pred) == normalize_sql(gold)

def execute_sql(sql: str, db_path: str):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        conn.close()
        return sorted(rows)
    except Exception as e:
        return f"[ERROR] {e}"

In [49]:
def evaluate_sql_quality(df, sql_col="predicted_sql", gold_col="gold_sql", db_path="db_path"):
    exact_matches = []
    execution_matches = []

    for _, row in df.iterrows():
        pred_sql = row[sql_col]
        gold_sql = row[gold_col]

        exact = exact_match(pred_sql, gold_sql)
        exact_matches.append(exact)

        pred_result = execute_sql(pred_sql, db_path)
        gold_result = execute_sql(gold_sql, db_path)
        exec_match = pred_result == gold_result
        execution_matches.append(exec_match)

    df["exact_match"] = exact_matches
    df["exec_match"] = execution_matches

    exact_acc = sum(exact_matches) / len(df)
    exec_acc = sum(execution_matches) / len(df)

    return {
        "exact_match_accuracy": round(exact_acc, 4),
        "execution_accuracy": round(exec_acc, 4)
    }

In [51]:
results = evaluate_sql_quality(temp, db_path="data/transactions.db")

In [54]:
print(results)

{'exact_match_accuracy': 0.23, 'execution_accuracy': 0.47}
